# Pretrained Models: ResNet50
Now that we know how to use a pretrained model, let's try to use a more complex model, ResNet50.

In [7]:
from torchvision.models import resnet50
import torch
import torchvision.transforms as T
from PIL import Image
import urllib.request

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
# load a sample image
img = Image.open("car.jpg").convert("RGB")

In [6]:
model = resnet50(pretrained=True).eval().to(device)

/home/ir-shar8/disruption/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ir-shar8/disruption/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/ir-shar8/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100.0%


In [9]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                std=[0.229, 0.224, 0.225])
])

In [10]:
# pass the image through the model
img_tensor = transform(img).unsqueeze(0).to(device)
with torch.no_grad():
    logits = model(img_tensor)
    probs = torch.nn.functional.softmax(logits, dim=1)


In [11]:
# Download the class labels
# Download the txt file with human-readable labels
url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
filename = "imagenet_classes.txt"
urllib.request.urlretrieve(url, filename)

# Load labels
with open(filename) as f:
    labels = [line.strip() for line in f.readlines()] # read all labels line by line

In [12]:
top5 = torch.topk(probs, 5) # get the top 5 probabilities and their indices
for i in range(5):
    class_id = top5.indices[0][i].item() # get the index of the top 5 classes
    score = top5.values[0][i].item() # get the score of the top 5 classes
    print(f"{i+1}. {labels[class_id]} ({score:.4f})")


1. beach wagon (0.4374)
2. grille (0.1133)
3. pickup (0.0982)
4. minivan (0.0504)
5. mobile home (0.0210)
